In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.uni-muenster.de/studium/orga/termine_archiv.html'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

ferien_datum = soup.find("tbody", class_="tab4")
table_rows = ferien_datum.find_all("tr")

data = []

for row in table_rows:
    columns = row.find_all("td")
    row_data = [column.get_text(strip=True) for column in columns]
    data.append(row_data)

print(data)


[['SoSe 2011', '01.04.2011', '04.04.2011', '14.06.2011', '18.06.2011', '15.07.2011', '30.09.2011'], ['WiSe 2011/12', '01.10.2011', '10.10.2011', '27.12.2011', '06.01.2012', '03.02.2012', '31.03.2012'], ['SoSe 2012', '01.04.2012', '02.04.2012', '29.05.2012', '02.06.2012', '13.07.2012', '30.09.2012'], ['WiSe 2012/13', '01.10.2012', '08.10.2012', '24.12.2012', '04.01.2013', '01.02.2013', '31.03.2013'], ['SoSe 2013', '01.04.2013', '08.04.2013', '21.05.2013', '25.05.2013', '19.07.2013', '30.09.2013'], ['WiSe 2013/14', '01.10.2013', '14.10.2013', '23.12.2013', '03.01.2014', '07.02.2014', '31.03.2014'], ['SoSe 2014', '01.04.2014', '07.04.2014', '10.06.2014', '14.06.2014', '18.07.2014', '30.09.2014'], ['WiSe 2014/15', '01.10.2014', '06.10.2014', '22.12.2014', '02.01.2015', '06.02.2015', '31.03.2015'], ['SoSe 2015', '01.04.2015', '07.04.2015', '26.05.2015', '29.05.2015', '17.07.2015', '30.09.2015'], ['WiSe 2015/16', '01.10.2015', '19.10.2015', '23.12.2015', '06.01.2016', '12.02.2016', '31.03.20

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://www.uni-muenster.de/studium/orga/termine_archiv.html'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

ferien_datum = soup.find("tbody", class_="tab4")
table_rows = ferien_datum.find_all("tr")

data = []

for row in table_rows:
    columns = row.find_all("td")
    row_data = [column.get_text(strip=True) for column in columns]
    data.append(row_data)

df_semesterferien = pd.DataFrame({'ferien': data})

column_names = ['semester', 'semester_beginn', 'vorlesungsbeginn', 'erster_ferientag', 
                'letzter_ferientag', 'vorlesungsende', 'semesterende']

df_semesterferien[column_names] = pd.DataFrame(df_semesterferien['ferien'].tolist(), columns=column_names)

date_columns = ['semester_beginn', 'vorlesungsbeginn', 'erster_ferientag', 
                'letzter_ferientag', 'vorlesungsende', 'semesterende']
for col in date_columns:
    df_semesterferien[col] = pd.to_datetime(df_semesterferien[col], format='%d.%m.%Y')

df_semesterferien.drop('ferien', axis=1, inplace=True)

df_semesterferien.to_csv('semesterferien.csv', index=False)

In [6]:
import pandas as pd

csv_file = 'semesterferien.csv'

df = pd.read_csv(csv_file)

data = []

for index, row in df.iterrows():
    semester = row['semester']
    semester_beginn = pd.to_datetime(row['semester_beginn'])
    vorlesungsbeginn = pd.to_datetime(row['vorlesungsbeginn'])
    erster_ferientag = pd.to_datetime(row['erster_ferientag'])
    letzter_ferientag = pd.to_datetime(row['letzter_ferientag'])
    vorlesungsende = pd.to_datetime(row['vorlesungsende'])
    semesterende = pd.to_datetime(row['semesterende'])
    
    current_date = vorlesungsbeginn
    while current_date <= vorlesungsende:
        data.append({'Semester': semester, 'Datum': current_date.date()})
        current_date += pd.DateOffset(days=1)

result_df = pd.DataFrame(data)

print(result_df)

          Semester       Datum
0        SoSe 2011  2011-04-04
1        SoSe 2011  2011-04-05
2        SoSe 2011  2011-04-06
3        SoSe 2011  2011-04-07
4        SoSe 2011  2011-04-08
...            ...         ...
2832  WiSe 2023/24  2024-01-29
2833  WiSe 2023/24  2024-01-30
2834  WiSe 2023/24  2024-01-31
2835  WiSe 2023/24  2024-02-01
2836  WiSe 2023/24  2024-02-02

[2837 rows x 2 columns]


In [7]:
result_df['Datum'] = pd.to_datetime(result_df['Datum']).dt.date
result_df

,Semester,Datum
0,SoSe 2011,2011-04-04
1,SoSe 2011,2011-04-05
2,SoSe 2011,2011-04-06
3,SoSe 2011,2011-04-07
4,SoSe 2011,2011-04-08
...,...,...
2832,WiSe 2023/24,2024-01-29
2833,WiSe 2023/24,2024-01-30
2834,WiSe 2023/24,2024-01-31
2835,WiSe 2023/24,2024-02-01


In [13]:
df_neutor = pd.read_csv('/Users/danielzellner/Documents/Studium/Bachelorthesis/src/verkehr_data/df_neutor.csv')

In [16]:
df_neutor['Semesterferien'] = 0
df_neutor.loc[df_neutor['Datum'].isin(result_df['Datum']), 'Semesterferien'] = 1

In [19]:
df_neutor.reset_index(drop=True, inplace=True)
df_neutor
print(df_neutor['Datum'].dtype)

object


In [18]:
anzahl_einsen = df_neutor["Semesterferien"].sum()

print(anzahl_einsen)

0
